# Ingestion de la carpeta "production_country"

In [0]:
dbutils.widgets.text("p_environment", "")
v_env = dbutils.widgets.get("p_environment")
v_env

In [0]:
%run "../includes/configuration"

In [0]:
%run "../includes/common_functions"

### Paso 1 - Leer los archivos CSV usando "DataFrameReader" de Spark

In [0]:
from pyspark.sql.types import StructType, StringType, StructField, IntegerType, DoubleType, StringType, DateType

In [0]:
production_countries_schema = StructType(fields=[
    StructField("movieId", IntegerType(), True),
    StructField("countryId", IntegerType(), True),
])

In [0]:
production_countries_df = spark.read.json(f'{bronze_folder_path}/production_country/', schema=production_countries_schema, multiLine=True)

In [0]:
production_countries_df.count()

In [0]:
production_countries_df.printSchema()

### Paso 2 - Renombrar columnas y adicionar columnas "ingestion_date" y "env"

In [0]:
from pyspark.sql.functions import col, lit, current_timestamp

In [0]:
production_countries_final_df = production_countries_df \
    .withColumnRenamed("movieId", "movie_id") \
    .withColumnRenamed("countryId", "country_id") \
    .transform(add_ingestion_date) \
    .withColumn('env',lit(v_env))

In [0]:
display(production_countries_final_df)

### Paso 3 - Escribir datos en el DataLake en formato parquet

In [0]:
# production_countries_final_df.write.mode('overwrite').parquet(f'{silver_folder_path}/production_country')
production_countries_final_df.write.mode('overwrite').format('parquet').saveAsTable('movie_silver.productions_countries')

In [0]:
%python
display(dbutils.fs.ls("/mnt/moviehistory/silver/productions_countries"))

In [0]:
df = spark.read.parquet('/mnt/moviehistory/silver/productions_countries')
display(df)

In [0]:
dbutils.notebook.exit("Sucess")